In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
from pathlib import Path
import os
import glob
from tqdm import tqdm

In [2]:
#Define category and subcategory based on url
directory = "D:/Darkweb Data/dnmarchives/hydra.tar/hydra/2014-10-27/category/"

subcat_nums = []
cat_nums = []

for i in os.listdir(directory):
    x = i[0:4]
    y = i[0:2]
    subcat_nums.append(x)
    cat_nums.append(y)
    
subcat_nums = sorted(set(subcat_nums))
cat_nums = sorted(set(cat_nums))

subcat_list = [
    'Cannabis',
    'Concentrates',
    'Hash',
    'Weed',
    'Synthetics',
    'Others',
    'Ecstasy',
    'MDMA',
    'Pills',
    'Others',
    'Opioids',
    'Heroin',
    'Others',
    'Dissociatives',
    'Ketamine',
    'GHB',
    'Others'
    'Psychedelics',
    'LSD',
    'DMT',
    'Mescaline',
    'Mushrooms',
    'Others',
    'Stimulants',
    'Cocaine',
    'Speed',
    'Meth',
    'Others',
    'Prescription',
    'Benzos',
    'Steroids',
    'Services',
    'Tabacco',
    'Weapons',
    'Arms',
    'Explosives',
    'Ammo',
    'Others',
    'Others',
    'Custom Orders',
    'Drug Paraphernalia',
    'Scales',
    'Rolling Supplies',
    'Reagent Tests',
    'Pipes',
    'Machinery',
    'Grinders',
    'Bongs',
    'Accessories',
    'Apparel',
    'Sunglasses',
    'Handbags',
    'Watches',
    'Others',
    'Digital Goods',
    'E-Books',
    'Money',
    'Erotica'
]

cat_list = [
    'Cannabis',
    'Ecstasy',
    'Opioids',
    'Dissociatives',
    'Psychedelics',
    'Stimulants',
    'Prescription',
    'Benzos',
    'Steroids',
    'Services',
    'Tabacco',
    'Weapons',
    'Others',
    'Custom Orders',
    'Drug Paraphernalia',
    'Apparel',
    'Digital Goods'
]

subcat_dict = dict(zip(subcat_nums, subcat_list))
cat_dict = dict(zip(cat_nums, cat_list))

In [3]:
# Define cleanhtml function
def cleanhtml(filename):
  """Extracts relevant information from Hydra html files. Post-5/08/14 format
  Resulting dataframe contains:
  Title, Seller ID, priceUSD, priceBTC, Rating, Reviews, Origin, Destination, Category, Market Name and Date Scraped"""
  with open(filename, encoding='utf8') as infile:  # Open file to support utf8 encoding
    raw_html = str(BeautifulSoup(infile, "html.parser"))
  cleanr = re.compile('<.*?>')  # Regex for html tags
  minustags = re.sub(cleanr, ' ', raw_html)  # Strip html tags
  nlines = re.compile(' ?\n ?')  # Regex for \n
  minusnline = re.sub(nlines, ' ', minustags)  # Strip nlines
  catlocations = re.compile('HYDRA.*?Price')  # Regex for majorite of header text, category/origin/destination options
  minuscatloc = re.sub(catlocations, '', minusnline)  # Strip html directory options 
  end = re.compile('( *&lt;.*?$)|(   Displaying.*?$)')  
  cleantext = re.sub(end, '', minuscatloc)
  attribute_list = re.split(r'\s{3,}', cleantext)
#  subcat = attribute_list[1].split(' :: ')[1].strip()  ##ONLY work on subcats
  title = attribute_list[3::8]  # Title of listing
  sellerid = attribute_list[4::8]  # Seller ID/Username
  rating = attribute_list[5::8]  # Seller Rating. Out of 5.
  reviews = attribute_list[6::8] # The number of customer reviews
  priceUSD = attribute_list[7::8] # Price in American Dollars
  priceBTC = attribute_list[8::8] # Price in Bitcoin
  origin = attribute_list[9::8]  # Origin of shipment
  destination = attribute_list[10::8]  # Where seller is willing to ship to
  category = attribute_list[1].split(' :: ')[0].strip() # The category of item
  category = [category] * len(title) # Fill column with single category value
  market = ['Hydra'] * len(title) # Fill column with single market name value
  date = [str(Path(filename).parents[1])[-10:]] * len(title)  # # Fill column with single date value
  df = pd.DataFrame([title,sellerid,priceUSD,priceBTC,rating,reviews,origin,destination,category,market,date]).transpose()  # Create df and transpose
  df.columns = ['Title','Sellerid','PriceUSD','PriceBTC','Rating','Reviews','Origin','Destination','Category','Market','Date']  # Assign column names to df
  return df

cleanhtml('D:/Darkweb Data/dnmarchives/hydra.tar/hydra/2014-10-24/category/1120.html.html') #test

,Title,Sellerid,PriceUSD,PriceBTC,Rating,Reviews,Origin,Destination,Category,Market,Date
0,** Maxs Wax - 1 gram**,OzGrow,$65.00,0.18006 BTC,5.0,6,Australia,Australia,Cannabis,Hydra,2014-10-24
1,"2,5 Gramm guter Marokkaner Hash (15€)",TanteEmma,$19.00,0.05263 BTC,5.0,27,Germany,Germany,Cannabis,Hydra,2014-10-24
2,2 oz ( 56gr ) ICE COOL best quality . Outdoor,BestWEEDseller,$278.00,0.77008 BTC,2.3,19,Undeclared,Worldwide,Cannabis,Hydra,2014-10-24
3,14g (1/2) NORTHERN LIGHTS TOP QUALITY!!!,BestWEEDseller,$110.00,0.30471 BTC,2.3,19,Undeclared,Worldwide,Cannabis,Hydra,2014-10-24
4,"50 grams Hash from marokko ""Louis Vuitton""",RedBull,$230.00,0.63712 BTC,5.0,325,Germany,European union,Cannabis,Hydra,2014-10-24
5,"20 gr hashish morrocan ""noir"" black",trikom,$120.00,0.33241 BTC,5.0,26,Netherlands,European union,Cannabis,Hydra,2014-10-24
6,Indian hand rubbed Charas 1g,enki,$17.00,0.04709 BTC,5.0,2,India,Worldwide,Cannabis,Hydra,2014-10-24
7,50 Gr nice quality blond HASH morrocan,trikom,$240.00,0.66482 BTC,5.0,26,Netherlands,European union,Cannabis,Hydra,2014-10-24
8,2.5 GR TO INDIAN HASH AAA+++ free shipment !,roquetas,$32.00,0.08864 BTC,4.7,117,Undeclared,Worldwide,Cannabis,Hydra,2014-10-24
9,1oz Malana Charas Cream! Imported direct from ...,FattusCatteratus,$300.00,0.83102 BTC,4.6,52,United States,United States,Cannabis,Hydra,2014-10-24


In [26]:
# Define cleanhtmlv2 function
def cleanhtmlv2(filename):
  """Extracts relevant information from Hydra html files. Post-5/08/14 format
  Resulting dataframe contains:
  Title, Seller ID, priceUSD, priceBTC, Rating, Reviews, Origin, Destination, Category, Market Name and Date Scraped"""
  with open(filename, encoding='utf8') as infile:  # Open file to support utf8 encoding
    raw_html = BeautifulSoup(infile, "lxml")
  title_raw = raw_html.find('tbody').find_all('a', {'class': 'bold btn btn-link'})
  cleantags = re.compile('<.*?>')  # Regex for html tags
  title = re.sub(cleantags, '', str(title_raw)).replace('[','').replace(']','').split(', ')
  category_raw = str(raw_html.find_all('title')).replace('[<title>','').replace('</title>]','')
  category_raw = str(category_raw) + ' :: None'
  category_full = category_raw.split(' :: ')
  selleridrating_raw = raw_html.find('tbody').find_all('a', {'class': 'btn btn-link'})
  selleridrating = re.sub(cleantags, '', str(selleridrating_raw)).replace('[','').replace(']','').replace(',','').split(' ')
  sellerid = selleridrating[0::5]  # Seller ID/Username
  rating = selleridrating[2::5]  # Seller Rating. Out of 5.
  reviews = selleridrating[4::5] # The number of customer reviews
  price_raw = raw_html.find('tbody').find_all('h5', {'class': 'text-success'})
  price_all = [i.text for i in price_raw]
  priceUSD = [i.split(' ', 1)[0] for i in price_all] # Price in American Dollars
  priceUSD = [i.replace('$','') for i in priceUSD]
  priceBTC = [i.split(' ', 2)[1] for i in price_all] # Price in Bitcoin
  shipping_raw = raw_html.find('tbody').find_all('td', {'class': 'col-xs-4'})  # Origin of shipment
  shipping_all = [i.text for i in shipping_raw]
  shipping = [i.split('BTC', 1)[1] for i in shipping_all]
  origin = [i.split('  ', 1)[0] for i in shipping]
  destination = [i.split('  ', 1)[1] for i in shipping]  # Where seller is willing to ship to
  category = [filename[64:66]] * len(title) 
  subcat = [filename[64:68]] * len(title)
  market = ['Hydra'] * len(title) # Fill column with single market name value
  date = [str(Path(filename).parents[1])[-10:]] * len(title)  # # Fill column with single date value
  df = pd.DataFrame([title,sellerid,priceUSD,priceBTC,rating,reviews,origin,destination,category,subcat,market,date]).transpose()  # Create df and transpose
  df.columns = ['Title','Sellerid','PriceUSD','PriceBTC','Rating','Reviews','Origin','Destination','Category','Subcategory','Market','Date']  # Assign column names to df
  df['Subcategory'] = df['Subcategory'].map(subcat_dict)
  df['Category'] = df['Category'].map(cat_dict)
  return df

cleanhtmlv2('D:/Darkweb Data/dnmarchives/hydra.tar/hydra/2014-08-09/category/1120.html.html') #test

,Title,Sellerid,PriceUSD,PriceBTC,Rating,Reviews,Origin,Destination,Category,Subcategory,Market,Date
0,15 gr Beautiful Creamy High Grade Pollen,Roquet,173.00,0.29545,5.0,1,Worldwide,Worldwide,Cannabis,Hash,Hydra,2014-08-09
1,8 GR to Beautiful Creamy High Grade Pollen,Roquet,107.00,0.18273,5.0,1,Worldwide,Worldwide,Cannabis,Hash,Hydra,2014-08-09
2,20 gr Beautiful Creamy High Grade Pollen,Roquet,204.00,0.34839,5.0,1,Worldwide,Worldwide,Cannabis,Hash,Hydra,2014-08-09
3,2.5 GR Beautiful Creamy High Grade Pollen,Roquet,38.00,0.0649,5.0,1,Worldwide,Worldwide,Cannabis,Hash,Hydra,2014-08-09
4,10 GR Beautiful Creamy High Grade Pollen,Roquet,129.00,0.22031,5.0,1,Worldwide,Worldwide,Cannabis,Hash,Hydra,2014-08-09
5,5 GR to Beautiful Creamy High Grade Pollen,Roquet,67.00,0.11442,5.0,1,Worldwide,Worldwide,Cannabis,Hash,Hydra,2014-08-09
6,1 gr to Beautiful Creamy High Grade Pollen,Roquet,19.00,0.03245,5.0,1,Worldwide,Worldwide,Cannabis,Hash,Hydra,2014-08-09
7,100gr Lion Hashish soft press Super Pollen (A+),DrEarnhardt,779.50,1.33123,5.0,1,Netherlands,Worldwide,Cannabis,Hash,Hydra,2014-08-09
8,50gr Lion Hashish soft press Super Pollen (A+),DrEarnhardt,422.00,0.72069,5.0,1,Netherlands,Worldwide,Cannabis,Hash,Hydra,2014-08-09
9,25gr Lion Hashish soft press Super Pollen (A+),DrEarnhardt,224.50,0.3834,5.0,1,Netherlands,Worldwide,Cannabis,Hash,Hydra,2014-08-09


In [5]:
# Define cleanhtml2 function for listings before 2014-08-09 but after 2014-04-03
def cleanhtml2(filename):
  """Extracts relevant information from Hydra html files. Pre-5/08/14, post-2014-04-03 format
  Resulting dataframe contains:
  Title, Seller ID, priceUSD, priceBTC, Rating, Reviews, Origin, Destination, Category, Market Name and Date Scraped"""
  with open(filename, encoding='utf8') as infile:  # Open file to support utf8 encoding
    raw_html = str(BeautifulSoup(infile, "html.parser"))
  cleanr = re.compile('<.*?>')  # Regex for html tags
  minustags = re.sub(cleanr, ' ', raw_html)  # Strip html tags
  nlines = re.compile(' ?\n ?')  # Regex for \n
  minusnline = re.sub(nlines, ' ', minustags)  # Strip nlines
  catlocations = re.compile('✧.*?Price')  # Regex for majorite of header text, category/origin/destination options
  minuscatloc = re.sub(catlocations, '', minusnline) # Strip html directory options 
  uppercategories = re.compile('Price.*?$')
  minusuppercats = re.sub(uppercategories, '', minuscatloc)
  attribute_list2 = re.split(r'\s{3,}', minusuppercats)
  category = attribute_list2[-2]  # The category of item
  end = re.compile(' *&lt;.*?$|←.*?$') 
  minusend = re.sub(end, '', minuscatloc)
  pprice = re.compile('^.*?Price')
  cleantext = re.sub(pprice, '', minusend)
  attribute_list = re.split(r'\s{2,}', cleantext)
  attribute_list = attribute_list[:(len(attribute_list)-len(attribute_list)%7)]
  title = attribute_list[2::7]  # Title of listing
  sellerid = attribute_list[3::7]  # Seller ID/Username
  rating = attribute_list[4::7]  # Seller Rating. Out of 5.
  reviews = [] * len(title)
  price = attribute_list[7::7] # Price
  priceUSD = [i.split(' ', 1)[0] for i in price]  #Extract first element of list of all price types (USD, BTC and sometimes LTC)
  priceUSD = [i.replace('$','') for i in priceUSD]
  priceBTC = [i.split(' ', 1)[0] for i in attribute_list[8::7]]
  priceBTC = [i.replace('฿','') for i in priceBTC]
  origin = attribute_list[5::7]  # Origin of shipment
  destination = attribute_list[6::7]  # Where seller is willing to ship to
  category = [filename[64:66]] * len(title) 
  subcat = [filename[64:68]] * len(title) 
  market = ['Hydra'] * len(title) # Fill column with single market name value
  date = [str(Path(filename).parents[1])[-10:]] * len(title)  # # Fill column with single date value
  df = pd.DataFrame([title,sellerid,priceUSD,priceBTC,rating,reviews,origin,destination,category,subcat,market,date]).transpose()  # Create df and transpose
  df.columns = ['Title','Sellerid','PriceUSD','PriceBTC','Rating','Reviews','Origin','Destination','Category','Subcategory','Market','Date']  # Assign column names to df
  df['Subcategory'] = df['Subcategory'].map(subcat_dict)
  df['Category'] = df['Category'].map(cat_dict)
  return df

cleanhtml2('D:/Darkweb Data/dnmarchives/hydra.tar/hydra/2014-08-05/category/1130.html.html') #test

,Title,Sellerid,PriceUSD,PriceBTC,Rating,Reviews,Origin,Destination,Category,Subcategory,Market,Date
0,"2 Grams of Prime Weed . (Wappa , Indica/Sativa...",HappyHippy2point0,30.00,0.05150745,5.0,None,United Kingdom,United Kingdom,Cannabis,Weed,Hydra,2014-08-05
1,28 Grams Great Bud,HeadacheCure,200.00,0.34338301,5.0,None,Canada,Worldwide,Cannabis,Weed,Hydra,2014-08-05
2,56 Grams Great Bud,HeadacheCure,400.00,0.68676602,5.0,None,Canada,Worldwide,Cannabis,Weed,Hydra,2014-08-05
3,112 Grams Great Bud,HeadacheCure,600.00,1.03014903,5.0,None,Canada,Worldwide,Cannabis,Weed,Hydra,2014-08-05
4,224 Grams Great Bud,HeadacheCure,1150.00,1.97445230,5.0,None,Canada,Worldwide,Cannabis,Weed,Hydra,2014-08-05
5,448 Grams Great Bud,HeadacheCure,2200.00,3.77721310,5.0,None,Canada,Worldwide,Cannabis,Weed,Hydra,2014-08-05
6,Amnesia Haze 10g (AA),QualityWeed,112.50,0.19315294,5.0,None,Netherlands,European Union,Cannabis,Weed,Hydra,2014-08-05
7,Amnesia Haze 10g (AAA+),QualityWeed,135.00,0.23178353,5.0,None,Netherlands,European Union,Cannabis,Weed,Hydra,2014-08-05
8,Quality Grown Weed -Northen Light-,VikingShrooms,12.00,0.02060298,5.0,None,Denmark,European Union,Cannabis,Weed,Hydra,2014-08-05
9,1lb Big Wreck (Big Bud x Trainwreck) Indoor,RogerTheAlien,None,None,5.0,None,United States,United States,Cannabis,Weed,Hydra,2014-08-05


In [6]:
# Define cleanhtmlv2 function
def cleanhtml2v2(filename):
  """Extracts relevant information from Hydra html files. Post-5/08/14 format
  Resulting dataframe contains:
  Title, Seller ID, priceUSD, priceBTC, Rating, Reviews, Origin, Destination, Category, Market Name and Date Scraped"""
  with open(filename, encoding='utf8') as infile:  # Open file to support utf8 encoding
    raw_html = BeautifulSoup(infile, "lxml")
  cleantags = re.compile('<.*?>')  # Regex for html tags
  title_raw = raw_html.find('tbody').find_all('a', {'class': 'btn btn-link bold'})
  title = [i.text for i in title_raw]#re.sub(cleantags, '', str(title_raw)).replace('[','').replace(']','').split(', ')
  category_raw = raw_html.find('h3').text
  category_raw = str(category_raw) + ' :: None'.replace('[<>','')
  category_full = category_raw.split(' :: ')
  selleridrating_raw = raw_html.find_all('a', {'class': 'btn btn-link'})
  selleridrating = re.sub(cleantags, '', str(selleridrating_raw)).replace('[','').replace(']','').replace(',','').split(' ')
  sellerid = selleridrating[1::4]  # Seller ID/Username
  rating = selleridrating[3::4]  # Seller Rating. Out of 5.
  reviews = [] # The number of customer reviews
  price_raw = raw_html.find('tbody').find_all('a', {'class': 'btn-block btn btn-success btn-small bold'})
  priceUSD_full = [i.text for i in price_raw]#[i.split(' ', 1)[0] for i in price_all] # Price in American Dollars
  priceUSD = [i.replace('$','') for i in priceUSD_full]
  priceBTC_raw = raw_html.find_all('p', {'class': 'text-center'})
  priceBTC_full = [i.text for i in priceBTC_raw]
  priceBTC = [i.replace('฿','') for i in priceBTC_full] # Price in Bitcoin
  shipping_raw = raw_html.find_all('p', {'class': 'text-left'})  
  shipping_full = re.sub(cleantags, ' ', str(shipping_raw)).replace('[','').replace(']','').split(', ')
  shipping = [i.split('    ', 3)[3] for i in shipping_full]
  origin = [i.split('    ', 1)[0] for i in shipping]  # Origin of shipment
  destination = [i.split('    ', 1)[1] for i in shipping]  # Where seller is willing to ship to. Needs trim.
  category = [filename[64:66]] * len(title) 
  subcat = [filename[64:68]] * len(title) 
  market = ['Hydra'] * len(title) # Fill column with single market name value
  date = [str(Path(filename).parents[1])[-10:]] * len(title)  # # Fill column with single date value
  df = pd.DataFrame([title,sellerid,priceUSD,priceBTC,rating,reviews,origin,destination,category,subcat,market,date]).transpose()  # Create df and transpose
  df.columns = ['Title','Sellerid','PriceUSD','PriceBTC','Rating','Reviews','Origin','Destination','Category','Subcategory','Market','Date']  # Assign column names to df
  return category_raw

#cleanhtml2v2('D:/Darkweb Data/dnmarchives/hydra.tar/hydra/2014-08-09/category/1110.html.html') #test
#cleanhtml2v2('D:/Darkweb Data/dnmarchives/hydra.tar/hydra/2014-08-05/category/1200.html.html') #test

In [7]:
# Define cleanhtml3 function
def cleanhtml3(filename):
  """Extracts relevant information from Hydra html files. Post-5/08/14 format
  Resulting dataframe contains:
  Title, Seller ID, priceUSD, priceBTC, Rating, Reviews, Origin, Destination, Category, Market Name and Date Scraped"""
  with open(filename, encoding='utf8') as infile:  # Open file to support utf8 encoding
    raw_html = BeautifulSoup(infile, "lxml")
  cleantags = re.compile('<.*?>')  # Regex for html tags
  title_raw = raw_html.find_all('a', {'class': 'bold btn btn-link'})
  title = [i.text for i in title_raw]#re.sub(cleantags, '', str(title_raw)).replace('[','').replace(']','').split(', ')
  category_raw = raw_html.find('title').text
  category_raw = str(category_raw) + ' :: None'.replace('[<>','')
  category_full = category_raw.split(' :: ')
  selleridrating_raw = raw_html.find_all('a', {'class': 'btn btn-link'})
  selleridrating = re.sub(cleantags, '', str(selleridrating_raw)).replace('[','').replace(']','').replace(',','').split(' ')
  sellerid = selleridrating[0::5]  # Seller ID/Username
  rating = selleridrating[2::5]  # Seller Rating. Out of 5.
  reviews = selleridrating[4::5] # The number of customer reviews for product
  shipping_raw = raw_html.find_all('td', {'class': 'col-xs-4'})  
  shipping_full = re.sub(cleantags, ' ', str(shipping_raw)).replace('[','').replace(']','').split(', ')
  shipping = [i.split('  ', 3)[3] for i in shipping_full]
  origin = [i.split('   ', 1)[0] for i in shipping]  # Origin of shipment
  destination = [i.split('    ', 1)[1] for i in shipping]  # Where seller is willing to ship to. Needs trim.
  price_raw =  [i.split('    ', 1)[0] for i in shipping_full]#[i.split(' ', 1)[0] for i in price_all] # Price in American Dollars
  priceUSD_full = [i.split('   ', 1)[0] for i in price_raw]
  priceUSD = [i.replace('  $','') for i in priceUSD_full]
  priceBTC_full = [i.split('   ', 1)[1] for i in price_raw]
  priceBTC = [i.replace(' BTC','') for i in priceBTC_full] # Price in Bitcoin
  category = [filename[64:66]] * len(title) 
  subcat = [filename[64:68]] * len(title) 
  market = ['Hydra'] * len(title) # Fill column with single market name value
  date = [str(Path(filename).parents[1])[-10:]] * len(title)  # # Fill column with single date value
  df = pd.DataFrame([title,sellerid,priceUSD,priceBTC,rating,reviews,origin,destination,category,subcat,market,date]).transpose()  # Create df and transpose
  df.columns = ['Title','Sellerid','PriceUSD','PriceBTC','Rating','Reviews','Origin','Destination','Category','Subcategory','Market','Date']  # Assign column names to df
  df['Subcategory'] = df['Subcategory'].map(subcat_dict)
  df['Category'] = df['Category'].map(cat_dict)
  return df

cleanhtml3('D:/Darkweb Data/dnmarchives/hydra.tar/hydra/2014-08-09/category/1200.html.html') #test

#Works on 2014-08-09

,Title,Sellerid,PriceUSD,PriceBTC,Rating,Reviews,Origin,Destination,Category,Subcategory,Market,Date
0,1 Gram PURE MDMA,dietcoke,55.00,0.09393,5.0,0,United States,United States,Ecstasy,Ecstasy,Hydra,2014-08-09
1,28g of MDMA US to US,RogerTheAlien,1200.00,2.04936,5.0,9,United States,United States,Ecstasy,Ecstasy,Hydra,2014-08-09
2,14g MDMA US to US,RogerTheAlien,650.00,1.11007,5.0,9,United States,United States,Ecstasy,Ecstasy,Hydra,2014-08-09
3,7g MDMA US to US,RogerTheAlien,330.00,0.56357,5.0,9,Worldwide,Worldwide,Ecstasy,Ecstasy,Hydra,2014-08-09
4,1g MDMA samplw,RogerTheAlien,40.00,0.06831,5.0,9,United States,United States,Ecstasy,Ecstasy,Hydra,2014-08-09
5,Pure MDMA (1oz = 28g),dietcoke,820.00,1.40039,5.0,0,United States,United States,Ecstasy,Ecstasy,Hydra,2014-08-09
6,Pure MDMA (14g),dietcoke,490.00,0.83682,5.0,0,United States,United States,Ecstasy,Ecstasy,Hydra,2014-08-09
7,Pure MDMA (7g),dietcoke,250.00,0.42695,5.0,0,United States,United States,Ecstasy,Ecstasy,Hydra,2014-08-09
8,Custom order H9,PROAMFETAMINE,180.00,0.3074,4.7,31,Germany,Worldwide,Ecstasy,Ecstasy,Hydra,2014-08-09
9,10x Ecstasy Pills (Green Euros) 220mg,thebear1928,130.00,0.22201,5.0,5,United States,United States,Ecstasy,Ecstasy,Hydra,2014-08-09


In [27]:
# Create empty variables to populate
#data = []
#data2 = []

# Specify file directory with folders for different scraping sessions
#datedir = "D:/Darkweb Data/dnmarchives/hydra.tar/hydra"

# Adds html ext to all files. Applies file cleaning function to all files from each scraping session and appends to one of two dataframes.
#for scrapedate in os.listdir(datedir):
#    directory = datedir +'/' + scrapedate + '/category/'
##    converthtml(directory, r'*.html*', r'%s')
#    for filename in os.listdir(directory):
#      file = directory + '/' + filename # Concatenate directory path and filename for reading into cleanhtml function
#      if scrapedate => '2014-08-09': #'2014-08-09':
#        filedf = cleanhtmlv2(file)  # Extract dataframe of required variables from each file
#        data.append(filedf)  # Append empty list with each dataframe generated. Results in list of dataframes.
#        df = pd.concat(data)  # Concatenate list of dataframes into single dataframe 
#      else:
#        filedf2 = cleanhtml2(file)
#        data2.append(filedf2) 
#        df2 = pd.DataFrame(data2)
#        df2 = pd.concat(data2)
        
        
# Create empty list to populate
data = []

# Specify file directory with folders for different scraping sessions
datedir = "D:/Darkweb Data/dnmarchives/hydra.tar/hydra"

# Adds html ext to all files. Applies file cleaning function to all files from each scraping session and appends to one of two dataframes.
for scrapedate in tqdm(os.listdir(datedir)):
    directory = datedir +'/' + scrapedate + '/category/'
    for filename in os.listdir(directory):
        file = directory + filename
        if scrapedate == '2014-05-08':
            filedf = cleanhtml3(file)  # Extract dataframe of required variables from each file
            data.append(filedf)  # Append empty list with each dataframe generated. Results in list of dataframes.
#        elif '2014-08-18' <= scrapedate:
#            filedf2 = cleanhtmlsr2part2(file)  # Extract dataframe of required variables from each file
#            data.append(filedf2)
df = pd.concat(data)  # Concatenate list of dataframes into single dataframe 

  0%|          | 0/35 [00:00<?, ?it/s]


IndexError: list index out of range

In [ ]:
df

In [ ]:
# Combine all dataframes
dataframe = df.append(df2).drop_duplicates()
dataframe

In [ ]:
# Check for inconsistencies
dataframe['Origin'].value_counts()[:40]

In [ ]:
# RDS info
#db name: cryptomarket-1
#username: dept922
#password: Daniel177

In [ ]:
# Saves data to PostgreSQL table
#engine = create_engine('postgresql://[USERNAME]:[PASSWORD]@[DATABASE].as-southeast-1.rds.amazonaws.com:5432/postgres', echo=True)
#df.to_sql('posting',engine, if_exists='replace', index = False)